In [1]:
import scipy as sp
from scipy import stats
import numpy as np
from numpy.lib.stride_tricks import as_strided

In [ ]:
x = np.array([1, 2, 3, 4], dtype=np.int8)

y = as_strided(x, strides=(0, 1*4), shape=(3, 4))
y
x.itemsize

In [ ]:
x = np.arange(100).reshape(10,10)
xSt = x.strides
x

In [ ]:
windows = as_strided(x, strides=(xSt[0], xSt[1],xSt[0], xSt[1] ), shape=(8,8,3,3))
windows[0,0,:,:]

In [ ]:
windows = as_strided(x, strides=(xSt[0], xSt[1],xSt[0], xSt[1] ), shape=(8,8,3,3))
windows.strides

In [ ]:
a = np.arange(125).reshape(5,5,5)

In [ ]:
s, i, r, p, st =  np.full((5,5), np.nan),np.full((5,5), np.nan),np.full((5,5), np.nan),np.full((5,5), np.nan),np.full((5,5), np.nan)

In [ ]:
stats.linregress(a[ind,ind,:], a[ind,ind,:]*2)


In [ ]:
def multiple_linregress(x, y):
    x_mean = np.mean(x, axis=2, keepdims=True)
    x_norm = x - x_mean
    y_mean = np.mean(y, axis=2, keepdims=True)
    y_norm = y - y_mean

    slope = (np.einsum('ijk,ijk->ij', x_norm, y_norm) /
             np.einsum('ijk,ijk->ij', x_norm, x_norm))
    del x_norm
    intercept = y_mean[:, :,0] - slope * x_mean[:, :,0]
    del x_mean
    y_pred_norm = (intercept[:,:,np.newaxis] + slope[:,:,np.newaxis] * x) - y_mean
    R2 = (np.einsum('ijk,ijk->ij', y_pred_norm, y_pred_norm) /
          np.einsum('ijk,ijk->ij', y_norm, y_norm))
    
    return R2, slope, intercept

m = 2500
n = 2500
x = np.random.rand(m, n, 9)
y = np.random.rand(m, n, 9)
r2,s,i = multiple_linregress(x, y)

In [81]:
m = 400
n = 400
x = np.random.rand(m, n, 3,3)
y = np.random.rand(m, n, 3,3)

x_mean = np.mean(x, axis=(2,3), keepdims=True)
x_norm = x - x_mean
y_mean = np.mean(y, axis=(2,3), keepdims=True)
y_norm = y - y_mean

slope = (np.einsum('ijkl,ijkl->ij', x_norm, y_norm) /
         np.einsum('ijkl,ijkl->ij', x_norm, x_norm))
del x_norm
intercept = y_mean[:, :,0,0] - slope * x_mean[:, :,0,0]
del x_mean
y_pred_norm = (intercept[:,:, np.newaxis, np.newaxis] + (slope[:,:, np.newaxis, np.newaxis] * x)) - y_mean
R2 = (np.einsum('ijkl,ijkl->ij', y_pred_norm, y_pred_norm) /
      np.einsum('ijkl,ijkl->ij', y_norm, y_norm))



In [114]:
def multiple_4d_linregress(x, y):
    x_mean = np.mean(x, axis=(2,3), keepdims=True)
    x_norm = x - x_mean
    y_mean = np.mean(y, axis=(2,3), keepdims=True)
    y_norm = y - y_mean

    slope = (np.einsum('ijkl,ijkl->ij', x_norm, y_norm) /
             np.einsum('ijkl,ijkl->ij', x_norm, x_norm))
    del x_norm
    intercept = y_mean[:, :,0,0] - slope * x_mean[:, :,0,0]
    del x_mean
    y_pred_norm = (slope[:,:, np.newaxis, np.newaxis] * x + intercept[:,:, np.newaxis, np.newaxis]) - y_mean
    R2 = (np.einsum('ijkl,ijkl->ij', y_pred_norm, y_pred_norm) /
          np.einsum('ijkl,ijkl->ij', y_norm, y_norm))
    
    return R2, slope, intercept

m = 100
n = 100
x = np.random.rand(m, n, 3,3)
y = np.random.rand(m, n, 3,3)

r2bis = np.full((m,n), np.nan)
sbis = np.full((m,n), np.nan)
ibis = np.full((m,n), np.nan)

start_time = timeit.default_timer()
r2,s,it = multiple_4d_linregress(x, y)
print(timeit.default_timer() - start_time)

r2bis = np.full((m,n), np.nan)
sbis = np.full((m,n), np.nan)
itbis = np.full((m,n), np.nan)
count = 0
start_time = timeit.default_timer()
for i in range(0,r2bis.shape[0]):
    for j in range(0,r2bis.shape[1]):
        sbis[i, j], itbis[i, j], r2bis[i, j] = sp.stats.linregress(x[i, j,:,:].flatten(), y[i, j,:,:].flatten())[:3]
print(timeit.default_timer() - start_time)

0.003057999230804853
1.7295443346793036


In [115]:
r2bis = r2bis**2

In [116]:
np.max(r2bis-r2)

7.7715611723760958e-16

In [51]:
import timeit
start_time = timeit.default_timer()
# code you want to evaluate
elapsed = timeit.default_timer() - start_time

In [3]:
x = np.arange(100).reshape(10,10)
windSh = (3, 3)
def GetWindows(x, windSh):
    xSt = x.strides
    xSh = x.shape
    return as_strided(x,
                      strides=(xSt[0], xSt[1], xSt[0], xSt[1] ),
                      shape=(xSh[0]-windSh[0]+1,xSh[1]-windSh[1]+1, 3, 3))
GetWindows(x, windSh).shape

(8, 8, 3, 3)

In [18]:
m = 100
n = 100
windSh = (3, 3)
x = np.random.rand(m, n)
y = np.random.rand(m, n)

r2 = np.full(x.shape, np.nan)
s = np.full(x.shape, np.nan)
i = np.full(x.shape, np.nan)

r2[1:-1, 1:-1],s[1:-1, 1:-1],i[1:-1, 1:-1] = multiple_4d_linregress(GetWindows(x, windSh), GetWindows(y, windSh))

In [19]:
r2bis = np.full(x.shape, np.nan)
sbis = np.full(x.shape, np.nan)
ibis = np.full(x.shape, np.nan)
count = 0
for i in range(1,r2bis.shape[0]-1):
    for j in range(1,r2bis.shape[1]-1):
        si, sj = slice(i-1,i+2), slice(j-1,j+2)
        sbis[i, j], ibis[i, j], r2bis[i, j] = sp.stats.linregress(x[si, sj].flatten(), y[si, sj].flatten())[:3]
        count = count + 1
    print(i)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98


In [28]:
sp.stats.linregress(x[si, sj].flatten(), y[si, sj].flatten())[:3]

(0.12348698373227515, 0.48055655718150408, 0.12321815139463094)

In [ ]:
i,j=0,0
width = 100
height = 100
i,j = i+100-wigth//2

In [ ]:
q,b =1,1

In [140]:

def my_OverlappingBlocks(x, y,width, heigth, winsize):
    
    assert x.shape == y.shape
    w, h = x.shape
    woverlape, hoverlape = winsize[0]-1, winsize[1]-1
        
    i, j = 0, 0
    while i+width < w: # iterate over rows
        while j+width < h: # iterate over column
            Si, Sj = slice(i,i+width), slice(j,j+heigth)
            Sio, Sjo = slice(i+1,i+width-1), slice(j+1,j+heigth-1)
            yield Sio, Sjo, x[Si, Sj], y[Si, Sj]

            j= j+heigth-hoverlape # actualize colum index 

        if j < h: # handle last columns
            Si, Sj = slice(i,i+width), slice(j,None)
            Sio, Sjo = slice(i+1,i+width-1), slice(j+1,-1)
            yield Sio, Sjo, x[Si, Sj], y[Si, Sj]
        i= i+width-woverlape # actualize row index
        j=0 # re-initialize column index
        
    if i  < w:  # handle last row
        while j+width < h:
            Si, Sj = slice(i,None), slice(j,j+heigth)
            Sio, Sjo = slice(i+1,-1), slice(j+1,j+heigth-1)
            yield Sio, Sjo, x[Si, Sj], y[Si, Sj]
            j= j+heigth-hoverlape

        if j < h: # handle last row, last line
            Si, Sj = slice(i,None), slice(j,None)
            Sio, Sjo = slice(i+1,-1), slice(j+1,-1)
            yield Sio, Sjo, x[Si, Sj], y[Si, Sj]

In [ ]:

x = np.arange(10000).reshape(100,100)
y = np.arange(10000).reshape(100,100)

iterator_test = my_OverlappingBlocks(x, y, width=10, heigth=10, overlape=2)
count =55
for Si, Sj, xx,yy in iterator_test:
    print( xx)
print(count)

In [ ]:
m = 10000
n = 10000
x = np.random.rand(m, n)
y = np.random.rand(m, n)
r2 = np.fill(x.shape, np.nan)
s = np.fill(x.shape, np.nan)
i = np.fill(x.shape, np.nan)

In [ ]:

def test(x, y):
    windSh = (3, 3)
    iterator_test = my_OverlappingBlocks(x, y, width=500, heigth=500, winsize=(3,3))
    count =55
    for Si, Sj, xx,yy in iterator_test:
        multiple_4d_linregress(GetWindows(xx, windSh), GetWindows(yy, windSh))
        count= count+1
    print(count)
%timeit test(x=x, y=y)

In [205]:
m = 100
n = 100
x = np.random.rand(m, n)
y = np.random.rand(m, n)
r2 = np.full(x.shape, np.nan)
s = np.full(x.shape, np.nan)
it = np.full(x.shape, np.nan)



In [145]:
windSh = (3, 3)
iterator_test = my_OverlappingBlocks(x, y, width=10, heigth=10, winsize=(3,3))
count =0

for Si, Sj, xx, yy in iterator_test:
    r2[Si, Sj], s[Si, Sj], it[Si, Sj] = multiple_4d_linregress(GetWindows(xx, windSh), GetWindows(yy, windSh))
    count = count +1
print(count)

169


In [210]:
def GetNeighboorRegress(x, y, blockSize, winSize, verbose=False):
    
    my_iterator = my_OverlappingBlocks(x,y,
                                       width=blockSize[0], 
                                       heigth=blockSize[1], 
                                       winsize=winSize)
    if verbose:
        count = 0
        for Si, Sj, xx, yy in my_iterator:
            
            r2[Si, Sj], s[Si, Sj], it[Si, Sj] = multiple_4d_linregress(GetWindows(xx, winSize), 
                                                                       GetWindows(yy, winSize))
            count = count +1
            print('Block: {4:d}, at location : [{0:d}:{1:d}] x [{2:d}:{3:d}]'.format(Si.start, Si.stop,
                                                                                     Sj.start, Sj.stop,
                                                                                    count))
    else:
        for Si, Sj, xx, yy in iterator_test:
            r2[Si, Sj], s[Si, Sj], it[Si, Sj] = multiple_4d_linregress(GetWindows(xx, winSize), 
                                                                       GetWindows(yy, winSize))
        
    return r2, s, it

In [211]:
GetNeighboorRegress(x, y, (10,10), (3,3), verbose=True)

Block: 1, at location : [1:9] x [1:9]
Block: 2, at location : [1:9] x [9:17]
Block: 3, at location : [1:9] x [17:25]
Block: 4, at location : [1:9] x [25:33]
Block: 5, at location : [1:9] x [33:41]
Block: 6, at location : [1:9] x [41:49]
Block: 7, at location : [1:9] x [49:57]
Block: 8, at location : [1:9] x [57:65]
Block: 9, at location : [1:9] x [65:73]
Block: 10, at location : [1:9] x [73:81]
Block: 11, at location : [1:9] x [81:89]
Block: 12, at location : [1:9] x [89:97]
Block: 13, at location : [1:9] x [97:-1]
Block: 14, at location : [9:17] x [1:9]
Block: 15, at location : [9:17] x [9:17]
Block: 16, at location : [9:17] x [17:25]
Block: 17, at location : [9:17] x [25:33]
Block: 18, at location : [9:17] x [33:41]
Block: 19, at location : [9:17] x [41:49]
Block: 20, at location : [9:17] x [49:57]
Block: 21, at location : [9:17] x [57:65]
Block: 22, at location : [9:17] x [65:73]
Block: 23, at location : [9:17] x [73:81]
Block: 24, at location : [9:17] x [81:89]
Block: 25, at locati

(array([[        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,  0.18406249,  0.15794652, ...,  0.00105579,
          0.45468925,         nan],
        [        nan,  0.12754176,  0.07089378, ...,  0.01318115,
          0.4247645 ,         nan],
        ..., 
        [        nan,  0.00947018,  0.39089559, ...,  0.26577847,
          0.15546691,         nan],
        [        nan,  0.11714565,  0.01931124, ...,  0.04702773,
          0.0224091 ,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan]]),
 array([[        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,  0.30611055,  0.33666917, ..., -0.04117039,
         -0.75060673,         nan],
        [        nan,  0.31689886,  0.21084825, ..., -0.14249175,
         -0.77764291,         nan],
        ..., 
        [        nan,  0.08201397, -0.39403619, ...,

In [204]:
xx

array([[ 0.11297921,  0.64668381,  0.48357886,  0.56933529],
       [ 0.08181166,  0.28078096,  0.07149813,  0.04618769],
       [ 0.16051848,  0.12553358,  0.70818043,  0.27128768],
       [ 0.39399264,  0.24135696,  0.74461613,  0.60753923]])

2

In [ ]:
np.isnan(r2[0:10,23:30])

In [146]:
r2bis = np.full(x.shape, np.nan)
sbis = np.full(x.shape, np.nan)
ibis = np.full(x.shape, np.nan)
count = 0
for i in range(1,x.shape[0]-1):
    for j in range(1,x.shape[1]-1):
        si, sj = slice(i-1,i+2), slice(j-1,j+2)
        sbis[i, j], ibis[i, j], r2bis[i, j] = sp.stats.linregress(x[si, sj].flatten(), y[si, sj].flatten())[:3]
        count = count + 1
    print(i)
r2bis = r2bis**2

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98


In [152]:
np.sum(np.logical_xor(np.isnan(r2bis),np.isnan(r2)))

0

In [ ]:
plt.imshow(np.isnan(r2bis))

In [ ]:
for i in range(1,x.shape[0]-1):
    for j in range(1,x.shape[1]-1):
        sp.stats.linregress(x[i-1:i+1,j-1:j+1].flatten(), y[i-1:i+1,j-1:j+1].flatten())

In [157]:
x = np.arange(10000).reshape(100,100)
y = np.arange(10000).reshape(100,100)
windSh = (3, 3)
iterator_test = my_OverlappingBlocks(x, y, width=5, heigth=5, winsize=windSh)
count =0
for Si, Sj, xx, yy in iterator_test:
    print(xx)


[[ 0.11448112  0.59734643  0.20622338  0.57295128  0.17356659]
 [ 0.73413185  0.76648432  0.86382032  0.15252061  0.57940833]
 [ 0.30660479  0.64210089  0.06091153  0.13173552  0.9392386 ]
 [ 0.84488522  0.63859029  0.02729909  0.16034582  0.7863367 ]
 [ 0.48600092  0.97046673  0.6044418   0.23375127  0.05211823]]
[[ 0.57295128  0.17356659  0.81310155  0.42684003  0.37062242]
 [ 0.15252061  0.57940833  0.21563661  0.50490765  0.88813824]
 [ 0.13173552  0.9392386   0.09499439  0.23305336  0.82719135]
 [ 0.16034582  0.7863367   0.29632348  0.4885983   0.06822974]
 [ 0.23375127  0.05211823  0.7658011   0.37789196  0.82157355]]
[[ 0.42684003  0.37062242  0.44606324  0.08626218  0.2965416 ]
 [ 0.50490765  0.88813824  0.23419429  0.34783364  0.20242336]
 [ 0.23305336  0.82719135  0.31559986  0.00515868  0.7111179 ]
 [ 0.4885983   0.06822974  0.56739676  0.56486883  0.36608814]
 [ 0.37789196  0.82157355  0.86445201  0.97287759  0.616745  ]]
[[ 0.08626218  0.2965416   0.50522172  0.45178474  0

 [ 0.68943673  0.56856295  0.68325177  0.31269756  0.35660382]]
[[ 0.86782456  0.85589178  0.17233578  0.86912359  0.38933281]
 [ 0.23516606  0.1548579   0.63938449  0.69979062  0.17008766]
 [ 0.56076385  0.79378178  0.55457198  0.92897117  0.43527059]
 [ 0.40968568  0.88002284  0.84566049  0.11457689  0.80999924]
 [ 0.31269756  0.35660382  0.21467831  0.87511939  0.34316883]]
[[ 0.86912359  0.38933281  0.34693866  0.52598814  0.91328645]
 [ 0.69979062  0.17008766  0.15454815  0.18766345  0.44433242]
 [ 0.92897117  0.43527059  0.21741992  0.97638352  0.29922258]
 [ 0.11457689  0.80999924  0.40742519  0.7654521   0.3970037 ]
 [ 0.87511939  0.34316883  0.60874646  0.05842658  0.70620149]]
[[ 0.52598814  0.91328645  0.58922104  0.15634086  0.54477988]
 [ 0.18766345  0.44433242  0.60972286  0.4710331   0.73746815]
 [ 0.97638352  0.29922258  0.22086976  0.29944125  0.90663719]
 [ 0.7654521   0.3970037   0.04556625  0.54237959  0.24344926]
 [ 0.05842658  0.70620149  0.27899712  0.37199251  0